## 1. Setup - Install Dependencies

In [ ]:
# Install ultralytics
!pip install ultralytics -q

# Verify GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Upload Dataset

**Option A: Upload ZIP file**
- First, create a ZIP of your yolo_data folder locally
- Then upload it using the file browser on the left

In [ ]:
# Create zip of yolo_data folder locally first:
# On Windows: Compress-Archive -Path yolo_data -DestinationPath yolo_data.zip
# Then upload yolo_data.zip to Colab

# Uncomment and run if you uploaded yolo_data.zip
# !unzip -q yolo_data.zip
# print("Dataset extracted!")

**Option B: Mount Google Drive**

If you uploaded yolo_data.zip to Google Drive:

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy dataset from Drive (adjust path as needed)
# !cp /content/drive/MyDrive/yolo_data.zip .
# !unzip -q yolo_data.zip

## 3. Verify Dataset Structure

In [ ]:
import os
from pathlib import Path

# Check dataset structure
data_path = Path('yolo_data')
print("Dataset structure:")
print(f"  Train images: {len(list((data_path / 'images' / 'train').glob('*')))}")
print(f"  Train labels: {len(list((data_path / 'labels' / 'train').glob('*')))}")
print(f"  Val images: {len(list((data_path / 'images' / 'val').glob('*')))}")
print(f"  Val labels: {len(list((data_path / 'labels' / 'val').glob('*')))}")

# Check data.yaml
import yaml
with open('yolo_data/data.yaml', 'r') as f:
    config = yaml.safe_load(f)
    print(f"\nClasses: {config['names']}")
    print(f"Number of classes: {config['nc']}")

## 4. Train YOLOv8 Model

Choose model size:
- `yolov8n.pt` - Nano (fastest, 3.2M params) - **Recommended for Colab free tier**
- `yolov8s.pt` - Small (11.2M params)
- `yolov8m.pt` - Medium (25.9M params)
- `yolov8l.pt` - Large (43.7M params) - Needs more GPU memory
- `yolov8x.pt` - Extra Large (68.2M params) - Needs premium GPU

In [ ]:
from ultralytics import YOLO

# Initialize model (downloads pretrained weights automatically)
model = YOLO('yolov8n.pt')  # Change to 's', 'm', 'l', or 'x' for larger models

# Train the model
results = model.train(
    data='yolo_data/data.yaml',
    epochs=50,              # More epochs = better accuracy (50-100 recommended)
    imgsz=640,              # Image size (640 is standard)
    batch=16,               # Batch size (reduce if GPU memory issues)
    device=0,               # Use GPU 0 (change to 'cpu' if no GPU)
    workers=2,              # Data loading workers
    patience=10,            # Early stopping patience
    save=True,              # Save checkpoints
    project='runs/detect',  # Project folder
    name='road_damage',     # Run name
    exist_ok=True,          # Overwrite existing
    pretrained=True,        # Use pretrained weights
    optimizer='AdamW',      # Optimizer
    lr0=0.001,              # Initial learning rate
    lrf=0.01,               # Final learning rate (lr0 * lrf)
    momentum=0.937,         # SGD momentum/Adam beta1
    weight_decay=0.0005,    # Optimizer weight decay
    warmup_epochs=3,        # Warmup epochs
    warmup_momentum=0.8,    # Warmup momentum
    box=7.5,                # Box loss gain
    cls=0.5,                # Class loss gain
    dfl=1.5,                # DFL loss gain
    hsv_h=0.015,            # HSV-Hue augmentation
    hsv_s=0.7,              # HSV-Saturation augmentation
    hsv_v=0.4,              # HSV-Value augmentation
    degrees=0.0,            # Rotation augmentation
    translate=0.1,          # Translation augmentation
    scale=0.5,              # Scale augmentation
    shear=0.0,              # Shear augmentation
    perspective=0.0,        # Perspective augmentation
    flipud=0.0,             # Vertical flip probability
    fliplr=0.5,             # Horizontal flip probability
    mosaic=1.0,             # Mosaic augmentation probability
    mixup=0.0,              # Mixup augmentation probability
    copy_paste=0.0,         # Copy-paste augmentation probability
    verbose=True,           # Verbose output
)

print("\n" + "="*50)
print("Training Complete!")
print("="*50)

## 5. Evaluate Model

In [ ]:
# Load best model
best_model = YOLO('runs/detect/road_damage/weights/best.pt')

# Validate on test set
metrics = best_model.val(data='yolo_data/data.yaml')

print("\n" + "="*50)
print("Validation Results:")
print("="*50)
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"\nPer-class AP@0.5:")
for i, ap in enumerate(metrics.box.ap50):
    class_name = ['Crack', 'Pothole', 'Severe_Damage'][i]
    print(f"  {class_name}: {ap:.4f}")

## 6. Test on Sample Images

In [ ]:
from IPython.display import Image, display
import glob

# Test on validation images
test_images = glob.glob('yolo_data/images/val/*.jpg')[:5]  # First 5 images

# Run inference
results = best_model.predict(test_images, save=True, conf=0.25)

# Display results
print("Sample predictions:")
for i, r in enumerate(results):
    print(f"\nImage {i+1}: {len(r.boxes)} detections")
    for box in r.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        class_name = ['Crack', 'Pothole', 'Severe_Damage'][cls]
        print(f"  - {class_name}: {conf:.2f}")

## 7. Export Model for Production

In [ ]:
# Export to multiple formats
best_model = YOLO('runs/detect/road_damage/weights/best.pt')

# Export to ONNX (for production)
best_model.export(format='onnx', dynamic=True, simplify=True)
print("✓ Exported to ONNX")

# Export to TorchScript (for PyTorch deployment)
best_model.export(format='torchscript')
print("✓ Exported to TorchScript")

# Export to TensorFlow Lite (for mobile)
try:
    best_model.export(format='tflite')
    print("✓ Exported to TensorFlow Lite")
except:
    print("⚠ TFLite export requires tensorflow package")

print("\nExported models are in: runs/detect/road_damage/weights/")

## 8. Download Trained Model

In [ ]:
# Zip all model files for download
!zip -r trained_model.zip runs/detect/road_damage/weights/

print("\n" + "="*50)
print("Model files ready for download!")
print("="*50)
print("\nDownload 'trained_model.zip' from the files panel on the left.")
print("\nContents:")
print("  - best.pt          (Best model checkpoint)")
print("  - last.pt          (Last epoch checkpoint)")
print("  - best.onnx        (ONNX format for production)")
print("  - best.torchscript (TorchScript format)")

# Or download directly (in Colab)
from google.colab import files
files.download('trained_model.zip')

## 9. View Training Results

In [ ]:
# Display training curves
from IPython.display import Image as IPImage

print("Training Results:")
results_images = [
    'runs/detect/road_damage/results.png',
    'runs/detect/road_damage/confusion_matrix.png',
    'runs/detect/road_damage/val_batch0_pred.png'
]

for img_path in results_images:
    if os.path.exists(img_path):
        print(f"\n{img_path}:")
        display(IPImage(filename=img_path, width=800))

## 10. Quick Inference Test (Optional)

In [ ]:
# Test on a single image URL (optional)
# Uncomment to test on a web image

# image_url = 'https://example.com/road-image.jpg'
# results = best_model.predict(image_url, save=True, conf=0.25)
# results[0].show()

---

## Next Steps:

1. **Download the trained model** (`trained_model.zip`)
2. **Extract it to your local project** (in the same folder as detect_realtime.py)
3. **Update model path** in `detect_realtime.py`:
   ```python
   model = YOLO('runs/detect/road_damage/weights/best.pt')
   ```
4. **Test real-time detection** locally:
   ```bash
   python detect_realtime.py --source 0 --api-key YOUR_GOOGLE_MAPS_KEY
   ```
5. **Deploy API server**:
   ```bash
   python api_server.py
   ```

## Training Tips:

- **Low mAP?** Increase epochs (100+), try larger model (yolov8s/m)
- **Overfitting?** Increase augmentation, reduce epochs
- **GPU memory issues?** Reduce batch size (8 or 4)
- **Slow training?** Use yolov8n, reduce image size (416 or 320)
- **Want better accuracy?** Use yolov8m or yolov8l with 100 epochs

**Expected Performance:**
- YOLOv8n: 88-90% mAP@0.5 (~30 min training)
- YOLOv8s: 90-92% mAP@0.5 (~45 min training)
- YOLOv8m: 92-94% mAP@0.5 (~60 min training)